In [34]:
%run 2.Ecoli_model.ipynb

Putrescine
ptrc_p
Putrescine
ptrc_c
Putrescine
ptrc_e
41.09128205128201
9.56477064220184
N-Acetyl-L-glutamate 5-semialdehyde
acg5sa_c
N2-Succinyl-L-glutamate 5-semialdehyde
sucgsa_c
L-Glutamate 5-semialdehyde
glu5sa_c
Ornithine
orn_e
Ornithine
orn_p
Ornithine
orn_c
2-Oxoglutarate
akg_e
2-Oxoglutarate
akg_c
2-Oxoglutarate
akg_p
L-Glutamate 1-semialdehyde
glu1sa_c
L-Glutamate
glu__L_c
L-Glutamate 5-semialdehyde
glu5sa_c
L-Glutamate
glu__L_e
L-Glutamate
glu__L_p
L-Glutamate 5-phosphate
glu5p_c
glu5sa_c + glu__L_c --> akg_c + orn_c
53.02333333333331
45.787428571428464
9.92914285714285
L-Arginine
arg__L_c
L-Arginine
arg__L_e
L-Arginine
arg__L_p
Urea CH4N2O
urea_c
Urea CH4N2O
urea_p
Urea CH4N2O
urea_e
H2O H2O
h2o_p
H2O H2O
h2o_c
H2O H2O
h2o_e
arg__L_c + h2o_c --> orn_c + urea_c
45.787428571428464
9.929142857142857


# Gene knock out strategies

In [35]:
#We would like to utilize the OptKnock built in algoritm in COBRApy
#First we make a list of essential genes, 
#second we make a list of non-essential genes

In [36]:
import cobra

In [37]:
def get_blocked_reactions(model):
    fva_res = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0)
    blocked = [
        model.reactions.get_by_id(reaction) for reaction, row in fva_res.data_frame.iterrows()
        if (round(row["lower_bound"], 5) == round(row["upper_bound"], 5) == 0)
    ]
    return blocked

In [38]:
get_blocked_reactions(model)

AttributeError: 'DataFrame' object has no attribute 'data_frame'

In [4]:
essentiality = {}
importantgenes = {}
targetgenes = {}
for gene in model.genes:
    with model:
        gene.knock_out()
        essentiality[gene] = model.slim_optimize(error_value=0.)
        #print(model.slim_optimize(error_value=0.))
        if model.slim_optimize(error_value=0.) == 0:
            importantgenes[gene] = model.slim_optimize(error_value=0.)
        else:
            targetgenes[gene] = model.slim_optimize(error_value=0.)
            

In [5]:
#importantgenes

In [6]:
importantgenes

{<Gene b3368 at 0x7fe3ec4070b8>: 0.0,
 <Gene b1260 at 0x7fe3ec3b7f98>: 0.0,
 <Gene b0109 at 0x7fe3ec38a710>: 0.0,
 <Gene b1094 at 0x7fe3ec38aa90>: 0.0,
 <Gene b2472 at 0x7fe3ec392128>: 0.0,
 <Gene b1261 at 0x7fe3ec392748>: 0.0,
 <Gene b3770 at 0x7fe3ec395b38>: 0.0,
 <Gene b0243 at 0x7fe3ec3974a8>: 0.0,
 <Gene b0414 at 0x7fe3ec397e10>: 0.0,
 <Gene b3041 at 0x7fe3ec3a04a8>: 0.0,
 <Gene b0025 at 0x7fe3ec3a07b8>: 0.0,
 <Gene b1662 at 0x7fe3ec3ac240>: 0.0,
 <Gene b0421 at 0x7fe3ec3ac898>: 0.0,
 <Gene b3804 at 0x7fe3ec3acbe0>: 0.0,
 <Gene b3997 at 0x7fe3ec3b0240>: 0.0,
 <Gene b3187 at 0x7fe3ec3b0860>: 0.0,
 <Gene b2400 at 0x7fe3ec3c2cf8>: 0.0,
 <Gene b3993 at 0x7fe3ec30c400>: 0.0,
 <Gene b1062 at 0x7fe3ec30bf28>: 0.0,
 <Gene b1281 at 0x7fe3ec30e898>: 0.0,
 <Gene b0639 at 0x7fe3ec30ef28>: 0.0,
 <Gene b0720 at 0x7fe3ec311278>: 0.0,
 <Gene b1136 at 0x7fe3ec311978>: 0.0,
 <Gene b2103 at 0x7fe3ec311fd0>: 0.0,
 <Gene b3642 at 0x7fe3ec328940>: 0.0,
 <Gene b0415 at 0x7fe3ec328f98>: 0.0,
 <Gene b3648

In [7]:
#targetgenes

In [8]:
#Make loop of our targetgenes list to append all reactions per gene

In [9]:
#targetgenes.index[0]

In [10]:
#model.genes.b2551.reactions
importantreactions = []
for gene in importantgenes:
    for reaction in gene.reactions:
        importantreactions.append(reaction)
importantreactions = list(set(importantreactions))
importantreactions



[<Reaction MECDPS at 0x7fe3ebdf8048>,
 <Reaction ACONIs at 0x7fe3eaf700b8>,
 <Reaction COBALT2tpp at 0x7fe3eacb80b8>,
 <Reaction GTPCI at 0x7fe3eb9c00f0>,
 <Reaction EAR80x at 0x7fe3ead70198>,
 <Reaction ASPO4 at 0x7fe3eb8801d0>,
 <Reaction DHDPRy at 0x7fe3ebc501d0>,
 <Reaction LPADSS at 0x7fe3ebec8208>,
 <Reaction MOADSUx at 0x7fe3eabd82b0>,
 <Reaction ENLIPAabctex at 0x7fe3eb8d02e8>,
 <Reaction CTPS2 at 0x7fe3ebd40358>,
 <Reaction ADSK at 0x7fe3ebf00390>,
 <Reaction NH4tpp at 0x7fe3eb5c03c8>,
 <Reaction GK1 at 0x7fe3eb8c84a8>,
 <Reaction GMPS at 0x7fe3ebf984a8>,
 <Reaction G3PAT120 at 0x7fe3ead704e0>,
 <Reaction DHNPA2r at 0x7fe3eaa50518>,
 <Reaction TMPPP at 0x7fe3ebed0550>,
 <Reaction 3OAS180 at 0x7fe3ead80550>,
 <Reaction DXPRIi at 0x7fe3ebd58588>,
 <Reaction PANTS at 0x7fe3eb1a85c0>,
 <Reaction G3PAT180 at 0x7fe3eae78630>,
 <Reaction AACPS6 at 0x7fe3eb1a86a0>,
 <Reaction ASPO6 at 0x7fe3eb8806a0>,
 <Reaction DHAD2 at 0x7fe3ebfc07f0>,
 <Reaction TDSK at 0x7fe3ebec8860>,
 <Reaction 

In [11]:
#Then we make a list of reactions we can knock out based on the knockable genes

In [12]:
#Then we run the algoritm for OptKnock

In [13]:
import cameo

In [14]:
from cameo.strain_design.deterministic.linear_programming import OptKnock

In [15]:
#for reaction in model.reactions.query('ATP', 'name'):
    #print(reaction.name)
    #print(reaction.id)

In [16]:
for reaction in model.reactions.query('CO2', 'name'):
    print(reaction.name)
    print(reaction.id)

CO2 exchange
EX_co2_e
CO2 transporter via diffusion (periplasm)
CO2tpp
CO2 transport via diffusion (extracellular to periplasm)
CO2tex


In [17]:
#for reaction in model.reactions.query('CO', 'name'):
  #  print(reaction.name)
   # print(reaction.id)

In [18]:
optknock = OptKnock(model, fraction_of_optimum=0.1, exclude_reactions= importantreactions)

ValueError: Excluded reaction ACONIs is not in the model

In [ ]:
optknock

In [ ]:
#result = optknock.run(max_knockouts=3, target="DM_ptrc_c", biomass="BIOMASS_Ec_iML1515_core_75p37M")

In [ ]:
model.reactions.COLIPAPabctex